In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys
sys.path

['/workspace/playground',
 '/opt/conda/lib/python310.zip',
 '/opt/conda/lib/python3.10',
 '/opt/conda/lib/python3.10/lib-dynload',
 '',
 '/home/docker_user/.local/lib/python3.10/site-packages',
 '/opt/conda/lib/python3.10/site-packages',
 '/opt/conda/lib/python3.10/site-packages/PyYAML-6.0-py3.10-linux-x86_64.egg']

In [4]:
import os
import random
import torch
import numpy as np
import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger
import argparse
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
import sys
import pandas as pd
# 일단 __init__.py 추가해서 이렇게 해놨음.
#BASE_PATH='/workspaces/kaggle-NFL-contact-detection'
BASE_PATH='/workspace'

sys.path.append(os.path.join(BASE_PATH, "src"))

# timm as third-party
sys.path.append(os.path.join(BASE_PATH, 'third-party/pytorch-image-models'))

from config import CFG
from factory.dataset_factory import DataSetFactory
from factory.lightning_module_factory import LightningModuleFactory
from train import seed_everything

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# G포함과 G미포함 인 경우로 나눠서 test dataset을 구성해보자.

In [6]:
import pytorch_lightning as pl
import random
import pandas as pd
import numpy as np
from torch.utils.data import random_split, Dataset, DataLoader, Subset
import cv2
import albumentations as A
from albumentations.pytorch import ToTensorV2
import os
from tqdm import tqdm
import glob
import subprocess
import torch
import gc
from typing import Tuple, List, Optional
import pickle

from config import CFG


class CNN25SingleGroundDataset(Dataset):
    def __init__(self, df, data_dir, preprocess_result_dir, feature_cols, video2frames, aug, mode='train'):
        self.df = df
        self.data_dir = data_dir
        # kaggle read only dir 문제로, 임시 생성 dir 별도 지정.
        self.preprocess_result_dir = preprocess_result_dir
        self.frame = df.frame.values
        self.feature = df[feature_cols].fillna(-1).values
        self.players = df[['nfl_player_id_1', 'nfl_player_id_2']].values
        self.game_play = df.game_play.values
        self.aug = aug
        self.mode = mode

        self.video2frames = video2frames
        self.image_size = 224
        os.makedirs(self.preprocess_result_dir, exist_ok=True)

    def __len__(self):
        return len(self.df)

    # @lru_cache(1024)
    # def read_img(self, path):
    #     return cv2.imread(path, 0)

    def __getitem__(self, idx):
        window = CFG["window"]  # 24
        frame = self.frame[idx]

        # TODO: 이 부분 의미 잘 모르겠음. (기존코드)
        # frame 값은 tracking data (10Hz)를 변환해서 얻은 값인데,
        # 이 값에 해당하는 정확한 frame 값이 helmet data에 없을 수 있음
        # if self.mode in ["fit", "validate", "test"]:
        #     frame = frame + random.randint(-6, 6)

        players = []
        for p in self.players[idx]:
            if p == 'G':
                players.append(p)
            else:
                players.append(str(p))

        imgs = []
        for view in ['Endzone', 'Sideline']:
            video = self.game_play[idx] + f'_{view}.mp4'
            bboxes = []
            query_game_play = self.game_play[idx]

            query_res = self.df.query(f"game_play == '{query_game_play}' and "
                                      f"nfl_player_id_1 == '{players[0]}' and "
                                      f"nfl_player_id_2 == '{players[1]}' and "
                                      f"frame == {frame}"
                                     )
            # bbox에 각 선수, view에 대한 frame의 window 평균 헬멧정보를 넣는다.
            for i in range(2):

                is_query_valid = False
                if len(query_res) == 1:
                    x = query_res.iloc[0][f"{view}_left_{i+1}"]
                    w = query_res.iloc[0][f"{view}_width_{i+1}"]
                    y = query_res.iloc[0][f"{view}_top_{i+1}"]
                    h = query_res.iloc[0][f"{view}_height_{i+1}"]
                    if all(not np.isnan(value) for value in [x, w, y, h]):
                        is_query_valid = True
                
                if is_query_valid:
                    bboxes.append([x, w, y, h])
                else:
                    bboxes.append([np.nan, np.nan, np.nan, np.nan])
            bboxes = pd.DataFrame(bboxes).interpolate(
                limit_direction='both').values
            bboxes = bboxes[::4]

            if bboxes.sum() > 0:
                flag = 1
            else:
                # 보통 트랙킹 데이터에서 가져온 선수가 해당 프레임에서 안잡힌 경우.
                flag = 0
                print(
                    f"flag: {flag}-{len(query_res)} , idx: {idx}, video: {video}, frame: {frame}, player: {players}")
# 0.03s

            # for i, f in enumerate(range(frame-window, frame+window+1, 4)):
            for i, f in enumerate(range(frame, frame+1, 4)):
                img_new = np.zeros(
                    (self.image_size, self.image_size, 3), dtype=np.float32)
                if f > self.video2frames[video]:
                    print(
                        f"flag: {flag}, frame: {f}, video frame: {self.video2frames[video]}")
                if flag == 1 and f <= self.video2frames[video]:
                    try:
                        img = cv2.imread(
                            os.path.join(self.preprocess_result_dir, f"frames/{video}_{f:04d}.jpg"), cv2.IMREAD_COLOR)
                        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                        
                        # 
                        
                        x, w, y, h = bboxes[i]
                        # print(f"img helmet size: {w} x {h}")
                        # 10~60 정도? helmet size
                        # 헬멧 크기의 10배 정도로 자름.
                        crop_size = int((max(w, h)*10))
                        # crop_size = 256
                        img_tmp = np.zeros(
                            (crop_size, crop_size, 3), dtype=np.float32)
                        # print(crop_size)
                        crop_half = crop_size // 2
                        crop_start_y = max(int(y+h/2)-crop_half, 0)
                        crop_end_y = min(int(y+h/2)+crop_half, img.shape[0])

                        crop_start_x = max(int(x+w/2)-crop_half, 0)
                        crop_end_x = min(int(x+w/2)+crop_half, img.shape[1])
                        # crop할 크기에 따라 원본 프레임에서 이미지 잘라냄.
                        img = img[crop_start_y:crop_end_y,
                                  crop_start_x:crop_end_x,
                                  :]
                        # 자른 후 정규화 및 transform을 추가한다.
                        img = self.aug(image=img)["image"]
                        

                        # 이미지 가장자리에서 crop 크기가 안나오는 경우 있음.
                        offset_y = (crop_size - img.shape[0]) // 2
                        offset_x = (crop_size - img.shape[1]) // 2
                        # resize 하기 전에 zero padding 사용해서 중앙으로 옮김.
                        img_tmp[offset_y: offset_y+img.shape[0],
                                offset_x: offset_x+img.shape[1],
                                :] = img

                        # resize를 통해 CNN에 들어갈 사이즈로 맞춰줌.
                        img_tmp = cv2.resize(
                            img_tmp, (self.image_size, self.image_size))
                        img_new[:img_tmp.shape[0], :img_tmp.shape[1], :] = img_tmp
                    except Exception as e:
                        print(os.path.join(self.preprocess_result_dir,
                              f"frames/{video}_{f:04d}.jpg"))
                        print(os.path.exists(os.path.join(
                            self.preprocess_result_dir, f"frames/{video}_{f:04d}.jpg")))
                        print(f"box: {(x,y)}, {(w,h)}")
                        print(f"img is None: {img is None}")
                        print(f"img shape: {img.shape}")
                        print(e)
                        raise e
                
                imgs.append(img_new)
# 0.06s

        feature = np.float32(self.feature[idx])
        imgs = np.array(imgs)
        imgs = imgs.transpose(0, 3, 1, 2)
        label = np.float32(self.df.contact.values[idx])

        return imgs, feature, label


class CNN25SingleGroundDataModule(pl.LightningDataModule):
    def __init__(self, data_dir: str = "./", preprocess_result_dir: str = "./"):
        super().__init__()
        self.data_dir = data_dir
        self.preprocess_result_dir = preprocess_result_dir

        self.train_aug = A.Compose([
            # Bright and Contrast가 의미 있는지 모르겠으나, normalize 안해주면 에러발생.
            A.ToFloat(max_value=255),
            A.HorizontalFlip(p=0.5),  # 숫자 뒤집어 짐
            A.ShiftScaleRotate(p=0.5, rotate_limit=5),  # 45도는 너무 큼.
            A.RandomBrightnessContrast(
                brightness_limit=(-0.1, 0.1), contrast_limit=(-0.1, 0.1), p=0.5),
            A.Normalize(mean=(0.485, 0.456, 0.406),
                        std=(0.229, 0.224, 0.225),
                        max_pixel_value=1.0),
        ])

        self.valid_aug = A.Compose([
            A.Normalize(mean=(0.485, 0.456, 0.406),
                        std=(0.229, 0.224, 0.225),
                        max_pixel_value=255.0)
        ])
        self.use_cols = [
            'x_position', 'y_position', 'speed', 'distance',
            'direction', 'orientation', 'acceleration', 'sa'
        ]
        # TODO: 이 부분 hard-coding 된 것 개선.
        self.feature_cols: List[str] = ["distance", "G_flug"]
        for col in self.use_cols:
            self.feature_cols.append(col + "_1")
            self.feature_cols.append(col + "_2")

        self.dataset_test = None
        self.dataset_train = None
        self.dataset_valid = None
        self.dataset_pred = None

    def expand_contact_id(self, df):
        """
        Splits out contact_id into seperate columns.
        """
        df["game_play"] = df["contact_id"].str[:12]
        df["step"] = df["contact_id"].str.split("_").str[-3].astype("int")
        df["nfl_player_id_1"] = df["contact_id"].str.split("_").str[-2]
        df["nfl_player_id_2"] = df["contact_id"].str.split("_").str[-1]
        return df

    def create_features(self, df, tr_tracking, merge_col="step", use_cols=["x_position", "y_position"]):
        output_cols = []
        df_combo = (
            df.astype({"nfl_player_id_1": "str"})
            .merge(
                tr_tracking.astype({"nfl_player_id": "str"})[
                    ["game_play", merge_col, "nfl_player_id",] + use_cols
                ],
                left_on=["game_play", merge_col, "nfl_player_id_1"],
                right_on=["game_play", merge_col, "nfl_player_id"],
                how="left",
            )
            .rename(columns={c: c+"_1" for c in use_cols})
            .drop("nfl_player_id", axis=1)
            .merge(
                tr_tracking.astype({"nfl_player_id": "str"})[
                    ["game_play", merge_col, "nfl_player_id"] + use_cols
                ],
                left_on=["game_play", merge_col, "nfl_player_id_2"],
                right_on=["game_play", merge_col, "nfl_player_id"],
                how="left",
            )
            .drop("nfl_player_id", axis=1)
            .rename(columns={c: c+"_2" for c in use_cols})
            .sort_values(["game_play", merge_col, "nfl_player_id_1", "nfl_player_id_2"])
            .reset_index(drop=True)
        )
        output_cols += [c+"_1" for c in use_cols]
        output_cols += [c+"_2" for c in use_cols]

        if ("x_position" in use_cols) & ("y_position" in use_cols):
            index = df_combo['x_position_2'].notnull()

            distance_arr = np.full(len(index), np.nan)
            tmp_distance_arr = np.sqrt(
                np.square(df_combo.loc[index, "x_position_1"] -
                          df_combo.loc[index, "x_position_2"])
                + np.square(df_combo.loc[index, "y_position_1"] -
                            df_combo.loc[index, "y_position_2"])
            )

            distance_arr[index] = tmp_distance_arr
            df_combo['distance'] = distance_arr
            output_cols += ["distance"]

        df_combo['G_flug'] = (df_combo['nfl_player_id_2'] == "G")
        output_cols += ["G_flug"]
        return df_combo, output_cols

    def prepare_data(self):
        # 다운로드 및 전처리
        # NOTE: single core로 실행됨.
        # https://pytorch-lightning.readthedocs.io/en/stable/data/datamodule.html
        # NOTE: state 저장하지 말고 disk에 저장해야함.
        self.preprocess_dataset()
        
    def reindex_by_frame(self, df):
        frames = np.arange(df.index.min()-CFG["window"], df.index.max()+CFG["window"])
        game_play = df.iloc[0]["game_play"]
        view = df.iloc[0]["view"]
        nfl_player_id = df.iloc[0]["nfl_player_id"]
        df = df.reindex(frames, fill_value=np.nan)
        df["game_play"] = df["game_play"].fillna(game_play)
        df["view"] = df["view"].fillna(view)
        df["nfl_player_id"] = df["nfl_player_id"].fillna(nfl_player_id)
        return df
    
    def preprocess_dataset(self):
        # 데이터 전처리 후, 파일로 저장해놓는다.
        # fit,validate,test 공통으로 한번, predict 한번씩만 실행되면 된다.
        is_prediction = CFG["is_prediction"]
        run_type = "test" if is_prediction else "train"
        frame_dir = os.path.join(self.preprocess_result_dir, "frames")
        processed_meta_dir = os.path.join(self.preprocess_result_dir, run_type)

        os.makedirs(frame_dir, exist_ok=True)
        os.makedirs(processed_meta_dir, exist_ok=True)

        print("====== [Preprocess] ======")
        print(f"- is_prediction: {is_prediction}")
        print(f"- run_type: {run_type}")

        print("------ [Loading Metadata] ------")
        df_helmets = pd.read_csv(os.path.join(
            self.data_dir, f"{run_type}_baseline_helmets.csv"))
        df_video_metadata = pd.read_csv(os.path.join(
            self.data_dir, f"{run_type}_video_metadata.csv"))

        print("------ [ffmpeg] ------")
        print(f"ffmpeg frames {run_type}")
        for video in tqdm(df_helmets.video.unique()):
            if os.path.isfile(os.path.join(frame_dir, video+"_0001.jpg")):
                continue
            if "Endzone2" not in video:
                subprocess.call(["ffmpeg", "-i", os.path.join(self.data_dir, f"{run_type}/{video}"), "-q:v", "2", "-f", "image2", os.path.join(
                    frame_dir, f"{video}_%04d.jpg"), "-hide_banner", "-loglevel", "error"])

        print("------ [Mapping metadata] ------")
#         if CFG["reproduce_processed_data"] or not os.path.exists(os.path.join(processed_meta_dir, "video2helmets.pickle")):
#             print(
#                 f"Mapping video2helmets [size: {len(df_helmets.video.unique())}]")
#             video2helmets = {}
#             df_helmets_new = df_helmets.set_index('video')
#             for video in tqdm(df_helmets.video.unique()):
#                 video2helmets[video] = df_helmets_new.loc[video].reset_index(
#                     drop=True)
#             with open(os.path.join(processed_meta_dir, "video2helmets.pickle"), "wb") as f:
#                 pickle.dump(video2helmets, f)
#             # 메모리 이슈
#             del df_helmets, df_helmets_new
#         else:
#             print(f"video2helemts already exists.. skip")

        if CFG["reproduce_processed_data"] or not os.path.exists(os.path.join(processed_meta_dir, "video2frames.pickle")):
            print(
                f"-- Mapping video2frames: [size: {len(df_video_metadata.game_play.unique())}]")
            video2frames = {}
            for game_play in tqdm(df_video_metadata.game_play.unique()):
                for view in ['Endzone', 'Sideline']:
                    video = game_play + f'_{view}.mp4'
                    video2frames[video] = max(list(map(
                        lambda x: int(x.split('_')[-1].split('.')[0]),
                        glob.glob(os.path.join(frame_dir, f'{video}*')
                                  ))))
                with open(os.path.join(processed_meta_dir, "video2frames.pickle"), "wb") as f:
                    pickle.dump(video2frames, f)
            # 메모리 이슈
            del video2frames
        else:
            print(f"video2frames already exists.. skip")

        gc.collect()

        print(f"------ [Preprocess helmet sensor data] ------")
        # CFG["reproduce_processed_data"]
        if CFG["reproduce_processed_data"] or not os.path.exists(os.path.join(processed_meta_dir, "df_filtered.csv")):
            df_tracking = pd.read_csv(os.path.join(
                self.data_dir, f"{run_type}_player_tracking.csv"))

            if is_prediction:
                label_file_name = "sample_submission.csv"
            else:
                label_file_name = "train_labels.csv"

            print(f"- Expand contact id")
            labels = self.expand_contact_id(pd.read_csv(
                os.path.join(self.data_dir, label_file_name)))
            print(f"- Create features")
            df_with_feature, _ = self.create_features(
                labels, df_tracking, use_cols=self.use_cols)
            df_filtered = df_with_feature.query(
                'not distance>2').reset_index(drop=True)
            df_filtered['frame'] = (
                df_filtered['step']/10*59.94+5*59.94).astype('int')+1

            # 메모리 이슈
            del df_with_feature, labels, df_tracking
            gc.collect()
            
            print(f"- Rolling helmet data")
            df_helmets["nfl_player_id"] = df_helmets["nfl_player_id"].astype("string")
            # helmet 데이터에 비어있는 frame을 nan으로 채운다.
            df_reindexed_helmets = df_helmets[["game_play","view", "frame", "nfl_player_id", "left","width", "top", "height"]]\
                .set_index("frame")\
                .groupby(["game_play", "view", "nfl_player_id"], dropna=False)\
                .apply(self.reindex_by_frame)\
                .reset_index(["game_play", "view", "nfl_player_id"], drop=True)
            
            # helmet 데이터의 각 frame을 window 크기로 돌며 위치와 크기를 평균낸다.
            df_rolled_helmets = df_reindexed_helmets.groupby(["game_play", "view", "nfl_player_id"], dropna=False)\
                .rolling(CFG["window"], min_periods=1, center=True)\
                .mean()\
                .reset_index()
            
            df_rolled_helmets["nfl_player_id"] = df_rolled_helmets["nfl_player_id"].astype("string")

            # Endzone2 있는거 제거
            df_rolled_helmets = df_rolled_helmets[df_rolled_helmets["view"] != "Endzone2"]

            # Endzone과 Sideline의 각 xwyh 를 분리한다.
            df_rolled_helmets = df_rolled_helmets.pivot_table(values=["left","width", "top", "height"], index=["game_play", "nfl_player_id", "frame"], columns=["view"], aggfunc="first").reset_index()

            # 컬럼 이름을 다시 지정한다.
            new_columns = []
            for c in df_rolled_helmets.columns.to_flat_index():
                if '' in c:
                    new_columns.append(c[0])
                else:
                    new_columns.append(c[1]+"_"+c[0])
            df_rolled_helmets.columns = new_columns
            # 각 컬럼이름
            rename_list = ["nfl_player_id", "Endzone_left", "Endzone_width", "Endzone_top", "Endzone_height", "Sideline_left", "Sideline_width", "Sideline_top", "Sideline_height"]

            # 각 컬럼 이름에 "_1"과 "_2"를 추가해서 rename으로 사용한다.
            for i in range(2):
                rename_dict = {k:k+f"_{i+1}" for k in rename_list}
                df_filtered[f"nfl_player_id_{i+1}"] = df_filtered[f"nfl_player_id_{i+1}"].astype("string")
                # player i+1에 대한 Enndzone, Sideline에서의 헬멧정보
                df_filtered = df_filtered.merge(df_rolled_helmets.rename(columns=rename_dict), 
                    left_on=["game_play", f"nfl_player_id_{i+1}", "frame"], 
                    right_on=["game_play", f"nfl_player_id_{i+1}", "frame"],
                    how="left"
                )

            # contact=1인데 헬멧정보가 없는 경우를 확인
            # print(df_filtered)
            # print(df_filtered[df_filtered["contact"] == 1 & (df_filtered["Endzone_left_1"].isnull() | df_filtered["Sideline_left_1"].isnull())])

            # save preprocessed files to writable dir.
            df_filtered.to_csv(os.path.join(
                processed_meta_dir, "df_filtered.csv"), index=False)
        else:
            print("df_filtered already exists.. skip")

    def generate_dataset(self, stage: str) -> CNN25SingleGroundDataset:
        # 학습 데이터 split을 수행한다.

        print(f"====== Generating dataset  ======")
        print(f"- stage: {stage}")

        if stage == "predict":
            run_type = "test"
        else:
            run_type = "train"

        processed_meta_dir = os.path.join(self.preprocess_result_dir, run_type)

        print(f"------ [Load metadata] ------")
        df_filtered = pd.read_csv(os.path.join(
            processed_meta_dir, f"df_filtered.csv"),
                                 dtype={"nfl_player_id_1":"string", "nfl_player_id_2":"string"})
        # type string
        
#         with open(os.path.join(processed_meta_dir, "video2helmets.pickle"), "rb") as f:
#             video2helmets = pickle.load(f)
        with open(os.path.join(processed_meta_dir, "video2frames.pickle"), "rb") as f:
            video2frames = pickle.load(f)

        if stage in ["fit", "validate", "test"]:
            # TODO: config 바로 이용하는게낫나, args로 넘기는게 낫나
            game_play_arr: np.ndarray
            game_play_arr = df_filtered.groupby(
                "game_play")["game_play"].first().to_numpy()
            # 사용할 game_play 수를 줄여준다.
            game_play_arr = game_play_arr[:CFG["num_train_video"]]

            train_video_size = int(len(game_play_arr) * 0.8)
            valid_video_size = int(len(game_play_arr) * 0.1)
            test_video_size = len(game_play_arr) - \
                (train_video_size + valid_video_size)
            # TODO: seed 고정을 함수 밖으로 옮기는게 나을지
            seed = torch.Generator().manual_seed(CFG["seed"])
            train_split, valid_split, test_split = random_split(
                game_play_arr, [train_video_size, valid_video_size, test_video_size], generator=seed)

            if stage == "fit":
                game_play_filter_arr = game_play_arr[train_split.indices]
            elif stage == "validate":
                game_play_filter_arr = game_play_arr[valid_split.indices]
            else:
                # test
                game_play_filter_arr = game_play_arr[test_split.indices]
            print(
                f"-- num_videos: {len(game_play_filter_arr)}/{len(game_play_arr)}")
            df_filtered_dataset = df_filtered[df_filtered["game_play"].isin(
                game_play_filter_arr)]
        else:
            df_filtered_dataset = df_filtered

        # NOTE: predict에서는 split 및 데이터 줄이는 과정이 필요없다.

        print(f"-- Label count {stage}: ")
        print(df_filtered_dataset.groupby("contact")["contact"].count())

        if stage in ["fit"]:
            aug = self.train_aug
        else:
            aug = self.valid_aug

        df_filtered_dataset = df_filtered_dataset.query(
            'G_flug == True').reset_index(drop=True)
        df_filtered_dataset = df_filtered_dataset.query(
            'contact == 1').reset_index(drop=True)

        dataset = CNN25SingleGroundDataset(
            df=df_filtered_dataset,
            data_dir=self.data_dir,
            preprocess_result_dir=self.preprocess_result_dir,
            feature_cols=self.feature_cols,
            video2frames=video2frames,
            aug=aug,
            mode=stage)

        return dataset

    def setup(self, stage: str):
        # dataset 생성
        # stage 는 fit/validate/test/predict 중 하나임.
        # train_ 데이터를 다시 train/validation/test로 나누고,
        # test_ 데이터는 predict에 사용함.

        # https://pytorch-lightning.readthedocs.io/en/stable/data/datamodule.html
        # 데이터셋 생성
        print("------ [Setup dataset] ------")
        if stage == "fit":
            self.dataset_train = self.generate_dataset("fit")
            print(f"- fit_dataset_size: {len(self.dataset_train)}")
            self.dataset_valid = self.generate_dataset("validate")
            print(f"- validate_dataset_size: {len(self.dataset_valid)}")
        elif stage == "validate":
            self.dataset_valid = self.generate_dataset("validate")
            print(f"- {stage}_dataset_size: {len(self.dataset_valid)}")
        elif stage == "test":
            self.dataset_test = self.generate_dataset("test")
            print(f"- {stage}_dataset_size: {len(self.dataset_test)}")
        elif stage == "predict":
            self.dataset_pred = self.generate_dataset("predict")
            print(f"- {stage}_dataset_size: {len(self.dataset_pred)}")
        else:
            raise TypeError("stage error.")

    def train_dataloader(self):
        return DataLoader(self.dataset_train, batch_size=CFG["batch_size"], num_workers=CFG["num_workers"], shuffle=True, drop_last=True)

    def val_dataloader(self):
        return DataLoader(self.dataset_valid, batch_size=CFG["batch_size"], num_workers=CFG["num_workers"])

    def test_dataloader(self):
        return DataLoader(self.dataset_test, batch_size=CFG["batch_size"], num_workers=CFG["num_workers"])

    def predict_dataloader(self):
        return DataLoader(self.dataset_pred, batch_size=CFG["batch_size"], num_workers=CFG["num_workers"])


In [7]:
# TODO: data dir train/pred 구분.
CFG["dataset_params"]["data_dir"] = os.path.join(BASE_PATH, "data")
CFG["dataset_params"]["preprocess_result_dir"] = os.path.join(BASE_PATH, "data/processed")
CFG["is_prediction"] = False
CFG["reproduce_processed_data"] = False
CFG["threshold"] = 0.344
CFG["num_workers"] = 4
CFG["window"]=24

In [8]:
seed_everything(CFG['seed'])
device_str = "cpu"
if torch.cuda.is_available():
    device_str = "cuda"
device_str

'cuda'

In [9]:
saved_model_path = os.path.join(
    BASE_PATH, 
    "epoch-8-step-198180-012222.ckpt"
)

In [10]:

model_name = f"{CFG['model_name']}-{CFG['model_version']}"
dataset_params = CFG['dataset_params']
model_params = CFG['model_params']
# data_module = DataSetFactory.get_dataset(name=model_name,
#                                              params=dataset_params)
data_module = CNN25SingleGroundDataModule(**dataset_params)
lightning_module = LightningModuleFactory.get_lightning_module(name=model_name,
                                                                load_path="",
                                                                params=model_params)

trainer = pl.Trainer(max_epochs=CFG["epochs"],
                        accelerator=device_str,
                        devices=1 if device_str != "cpu" else None,
                        logger=False
                    )

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [11]:
import matplotlib.pyplot as plt
data_module.prepare_data()
data_module.setup("test")
dl = data_module.test_dataloader()

====== [Preprocess] ======
- is_prediction: False
- run_type: train
------ [Loading Metadata] ------
------ [ffmpeg] ------
ffmpeg frames train


100%|█████████████████████████████████████| 481/481 [00:00<00:00, 257657.76it/s]

------ [Mapping metadata] ------
video2frames already exists.. skip


------ [Preprocess helmet sensor data] ------
df_filtered already exists.. skip
------ [Setup dataset] ------
====== Generating dataset  ======
- stage: test
------ [Load metadata] ------
-- num_videos: 24/240
-- Label count test: 
contact
0    58681
1     6696
Name: contact, dtype: int64
- test_dataset_size: 2006


In [12]:
df = data_module.dataset_test.df
print(len(df))
print(len(df[df["contact"] == 1 & df["Endzone_left_1"].isnull()]))
print(len(df[df["contact"] == 1 & df["Sideline_left_1"].isnull()]))
print(len(df[df["contact"] == 1 & df["Endzone_left_1"].isnull() & df["Sideline_left_1"].isnull()]))
print(len(df[df["contact"] == 1 & ~df["Endzone_left_1"].isnull() & ~df["Sideline_left_1"].isnull()]))
print(336 + 249 - 147 + 1568)
df[df["contact"] == 1 & ~df["Endzone_left_1"].isnull() & ~df["Sideline_left_1"].isnull()]

2006
336
249
147
1568
2006


,contact_id,game_play,datetime,step,nfl_player_id_1,nfl_player_id_2,contact,x_position_1,y_position_1,speed_1,...,Endzone_width_1,Sideline_width_1,Endzone_height_2,Sideline_height_2,Endzone_left_2,Sideline_left_2,Endzone_top_2,Sideline_top_2,Endzone_width_2,Sideline_width_2
0,58187_001383_72_38696_G,58187_001383,2020-09-20T17:56:22.500Z,72,38696,G,1,77.86,48.38,5.71,...,19.708333,9.444444,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,58187_001383_72_46087_G,58187_001383,2020-09-20T17:56:22.500Z,72,46087,G,1,77.40,47.97,6.00,...,23.916667,13.208333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,58187_001383_73_38696_G,58187_001383,2020-09-20T17:56:22.600Z,73,38696,G,1,78.20,48.79,5.10,...,21.333333,11.388889,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,58187_001383_73_46087_G,58187_001383,2020-09-20T17:56:22.600Z,73,46087,G,1,77.78,48.41,5.55,...,25.541667,14.166667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,58187_001383_74_38696_G,58187_001383,2020-09-20T17:56:22.700Z,74,38696,G,1,78.50,49.13,4.22,...,23.708333,12.047619,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1946,58565_001768_59_25511_G,58565_001768,2021-10-04T01:27:50.900Z,59,25511,G,1,37.00,21.22,0.27,...,14.727273,7.857143,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1948,58565_001768_60_25511_G,58565_001768,2021-10-04T01:27:51.000Z,60,25511,G,1,37.01,21.15,0.40,...,14.583333,8.250000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1950,58565_001768_61_25511_G,58565_001768,2021-10-04T01:27:51.100Z,61,25511,G,1,36.99,21.11,0.49,...,13.600000,8.444444,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1952,58565_001768_62_25511_G,58565_001768,2021-10-04T01:27:51.200Z,62,25511,G,1,36.96,21.05,0.59,...,10.000000,8.666667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
q = df.query("game_play == '58187_001383' and nfl_player_id_1 == '38696' and frame == 732")
len(q)
df.iloc[250]

contact_id            58204_002864_31_42486_G
game_play                        58204_002864
datetime             2020-09-27T19:18:16.100Z
step                                       31
nfl_player_id_1                         42486
nfl_player_id_2                             G
contact                                     1
x_position_1                            110.4
y_position_1                            28.16
speed_1                                  0.86
distance_1                               0.09
direction_1                            147.31
orientation_1                          344.54
acceleration_1                           1.45
sa_1                                    -1.44
x_position_2                              NaN
y_position_2                              NaN
speed_2                                   NaN
distance_2                                NaN
direction_2                               NaN
orientation_2                             NaN
acceleration_2                    

In [ ]:
mean=torch.tensor([0.485, 0.456, 0.406])
std=torch.tensor([0.229, 0.224, 0.225])
for r, d in enumerate(dl):
    print(r)
    imgs, feature, label = d
    print(imgs.size())
    imgs1, imgs2 = torch.split(imgs, 1, dim=1)
    print(imgs1.size(), imgs2.size())
    batch_size = imgs1.size()[0]


    num_imgs = int(imgs1.size()[0])
    # f, axarr = plt.subplots(2, num_imgs, figsize=(100,30))
    f, axarr = plt.subplots(4, num_imgs // 2, figsize=(100, 30))
    for i in range(num_imgs // 2):
        img1, img2 = imgs1[i], imgs2[i]
        axarr[0,i].imshow(img1[0].permute(1,2,0)*std+mean)  # end
        axarr[1,i].imshow(img2[0].permute(1,2,0)*std+mean)  # side
    for i in range(num_imgs // 2):
        img1, img2 = imgs1[num_imgs // 2 + i], imgs2[num_imgs // 2 + i]
        axarr[2,i].imshow(img1[0].permute(1,2,0)*std+mean)  # end
        axarr[3,i].imshow(img2[0].permute(1,2,0)*std+mean)  # side
        
    plt.show()
    if r >= 0:
        break
plt.close()

In [149]:
i = cv2.imread("/workspace/data/processed/frames/58204_002864_Endzone.mp4_0390.jpg", 1)

torch.Size([3, 224, 224])

In [155]:
mean=torch.tensor([0.485, 0.456, 0.406])
std=torch.tensor([0.229, 0.224, 0.225])
img1[0].permute(1,2,0)*std + mean

tensor([[[0.3373, 0.4431, 0.2275],
         [0.3373, 0.4431, 0.2275],
         [0.3373, 0.4431, 0.2275],
         ...,
         [0.7765, 0.7608, 0.7137],
         [0.7767, 0.7608, 0.7137],
         [0.7843, 0.7608, 0.7137]],

        [[0.3411, 0.4508, 0.2236],
         [0.3411, 0.4471, 0.2311],
         [0.3411, 0.4506, 0.2240],
         ...,
         [0.7803, 0.7646, 0.7175],
         [0.7805, 0.7646, 0.7175],
         [0.7881, 0.7646, 0.7175]],

        [[0.3449, 0.4547, 0.2235],
         [0.3449, 0.4546, 0.2275],
         [0.3449, 0.4547, 0.2237],
         ...,
         [0.7804, 0.7647, 0.7176],
         [0.7804, 0.7647, 0.7176],
         [0.7807, 0.7647, 0.7176]],

        ...,

        [[0.2980, 0.4314, 0.1922],
         [0.3057, 0.4276, 0.1922],
         [0.3021, 0.4237, 0.1884],
         ...,
         [0.5649, 0.6276, 0.5257],
         [0.5619, 0.6244, 0.5225],
         [0.5881, 0.6430, 0.5449]],

        [[0.2980, 0.4314, 0.1922],
         [0.3057, 0.4276, 0.1922],
         [0.